# Finanční analýza s využitím GraphQL, JSON a Excel

Tento notebook ukazuje proces získání dat z GraphQL API, jejich úpravu a následné zobrazení ve formě grafů v Excelu.

## Instalace potřebných balíčků

Nejprve nainstalujeme všechny potřebné balíčky.

In [1]:
!pip install aiohttp pandas openpyxl plotly kaleido

## Načtení dat z GraphQL

Zde definujeme funkce pro získání tokenu, dotazování se na GraphQL a transformaci dat do formátu JSON.

In [2]:
import asyncio
import aiohttp
import pandas as pd
import json

# Asynchronní funkce pro získání tokenu
async def getToken(username, password):
    keyurl = "http://localhost:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            if resp.status != 200:
                raise Exception(f"Failed to get key: {resp.status}")
            keyJson = await resp.json()

        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            if resp.status != 200:
                raise Exception(f"Failed to get token: {resp.status}")
            tokenJson = await resp.json()

    return tokenJson.get("token", None)

# Funkce pro vytvoření dotazu na GraphQL API
def query(q, token):
    async def post(variables):
        gqlurl = "http://localhost:33001/api/gql"
        payload = {"query": q, "variables": variables}
        cookies = {'authorization': token}

        async with aiohttp.ClientSession() as session:
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                if resp.status != 200:
                    text = await resp.text()
                    raise Exception(f"Query failed: {resp.status} - {text}")
                return await resp.json()
    return post

username = "john.newbie@world.com"
password = "john.newbie@world.com"
queryStr = """
query FinanceAnalysis {
  financePage {
    id
    name
    amount
    valid
    lastchange
    financeType {
      id
      name
    }
    project {
      id
      name
      startdate
      enddate
      valid
      team {
        id
        name
      }
    }
    changedby {
      id
      name
    }
  }
}
"""

# Funkce pro transformaci dat z GraphQL do formátu JSON
def transform_gql_to_json(gql_query):
    sourceTable = []

    for finance_item in gql_query["financePage"]:
        row = {}
        row["id"] = finance_item["id"]
        row["name"] = finance_item["name"]
        row["amount"] = finance_item["amount"]
        row["valid"] = finance_item["valid"]
        row["lastchange"] = finance_item["lastchange"]
        row["financeTypeName"] = finance_item["financeType"][0]["name"]
        row["projectID"] = finance_item["project"]["id"]
        row["projectName"] = finance_item["project"]["name"]
        row["projectStartDate"] = finance_item["project"]["startdate"]
        row["projectEndDate"] = finance_item["project"]["enddate"]
        row["projectValid"] = finance_item["project"]["valid"]
        row["teamID"] = finance_item["project"]["team"]["id"]
        row["teamName"] = finance_item["project"]["team"]["name"]
        row["changedby"] = finance_item.get("changedby")  # Ensure changedby exists
        
        sourceTable.append(row)

    return sourceTable

# Asynchronní funkce pro provedení celého procesu
async def fullPipe():
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})

    data = response.get("data")
    if not data:
        raise ValueError("No data found in the response.")
    
    transformed_data = transform_gql_to_json(data)
    pandasData = pd.DataFrame(transformed_data)
    return pandasData

loop = asyncio.get_event_loop()
try:
    pandasData = loop.run_until_complete(fullPipe())
    print(pandasData)
    pandasData.to_json('finance_analysis.json', orient='records', lines=True)
except Exception as e:
    print(f"An error occurred: {e}")

## Přidání dodatečných dat do JSON

Následující buňka přidá další data do JSON souboru.

In [3]:
import json
import os

# Funkce pro přidání dodatečných dat do existujícího JSON souboru
def add_additional_data(json_file):
    additional_data = [
        {
            "id": "f911230f-7e1f-4e9b-90a9-b921996ceb88",
            "name": "komplet2",
            "amount": 200000,
            "valid": False,
            "lastchange": "2024-06-07T13:41:14.491289",
            "financeTypeName": "firemní náklady",
            "projectID": "43dd2ff1-5c17-42a5-ba36-8b30e2a243bc",
            "projectName": "Nukleární reaktor pro budovy 2",
            "projectStartDate": "2021-01-01T17:27:12",
            "projectEndDate": "2022-12-31T17:27:12",
            "projectValid": True,
            "teamID": "2d9dcd22-a4a2-11ed-b9df-0242ac120004",
            "teamName": "Uni 2",
            "changedby": None
        },
        {
            "id": "f911230f-7e1f-4e9b-90a9-b921996ceb89",
            "name": "komplet3",
            "amount": 300000,
            "valid": True,
            "lastchange": "2024-06-08T13:41:14.491289",
            "financeTypeName": "dopravní náklady",
            "projectID": "43dd2ff1-5c17-42a5-ba36-8b30e2a243bd",
            "projectName": "Nukleární reaktor pro budovy 3",
            "projectStartDate": "2026-01-01T17:27:12",
            "projectEndDate": "2027-03-20T17:27:12",
            "projectValid": True,
            "teamID": "2d9dcd22-a4a2-11ed-b9df-0242ac120005",
            "teamName": "Uni 3",
            "changedby": None
        },
        {
            "id": "f911230f-7e1f-4e9b-90a9-b921996ceb90",
            "name": "komplet4",
            "amount": 400000,
            "valid": True,
            "lastchange": "2024-06-09T13:41:14.491289",
            "financeTypeName": "technické náklady",
            "projectID": "43dd2ff1-5c17-42a5-ba36-8b30e2a243be",
            "projectName": "Nukleární reaktor pro budovy 4",
            "projectStartDate": "2024-01-01T17:27:12",
            "projectEndDate": "2024-11-29T17:27:12",
            "projectValid": True,
            "teamID": "2d9dcd22-a4a2-11ed-b9df-0242ac120006",
            "teamName": "Uni 4",
            "changedby": None
        }
    ]

    with open(json_file, 'r') as file:
        data = json.load(file)

    if isinstance(data, dict):  
        data = [data]
    data.extend(additional_data)

    with open(json_file, 'w') as file:
        json.dump(data, file, indent=4)

json_file = "finance_analysis.json"
if os.path.exists(json_file):
    add_additional_data(json_file)
    print("Dodatečná data byla úspěšně přidána.")
else:
    print("Soubor finance_analysis.json neexistuje.")

## Zpracování dat do Excelu

Nakonec zpracujeme data a uložíme je do Excelu včetně grafů.

In [4]:
import json
import os
import pandas as pd
import openpyxl
from openpyxl.chart import PieChart, LineChart, Reference, Series
from openpyxl.utils.dataframe import dataframe_to_rows  
import plotly.express as px

# Funkce pro načtení JSON souboru
def nacti_json_soubor(soubor):
    """Načte JSON soubor a vrátí jeho obsah."""
    with open(soubor, 'r', encoding='utf-8') as soubor:
        return json.load(soubor)

# Funkce pro převod JSON dat na DataFrame
def preved_json_na_dataframe(data):
    """Převede data z JSON na DataFrame."""
    zpracovana_data = {
        'ID': [],
        'Název': [],
        'Částka': [],
        'Typ financí': [],
        'Název projektu': [],
        'Datum zahájení': [],
        'Datum ukončení': []
    }

    for stranka in data:
        zpracovana_data['ID'].append(stranka['id'])
        zpracovana_data['Název'].append(stranka['name'])
        zpracovana_data['Částka'].append(stranka['amount'])
        zpracovana_data['Typ financí'].append(stranka['financeTypeName'])
        zpracovana_data['Název projektu'].append(stranka['projectName'])
        zpracovana_data['Datum zahájení'].append(stranka['projectStartDate'])
        zpracovana_data['Datum ukončení'].append(stranka['projectEndDate'])

    return pd.DataFrame(zpracovana_data)

# Funkce pro uložení DataFrame do Excel souboru včetně grafů
def uloz_do_excelu(dataframe, excel_soubor):
    """Uloží DataFrame do Excel souboru včetně grafů."""
    wb = openpyxl.Workbook()
    ws = wb.active

    for radek in dataframe_to_rows(dataframe, index=False, header=True):
        ws.append(radek)

    # Vytvoř koláčový graf
    kolac = PieChart()
    popisky = Reference(ws, min_col=4, min_row=2, max_row=len(dataframe)+1)
    data = Reference(ws, min_col=3, min_row=2, max_row=len(dataframe)+1)
    kolac.add_data(data, titles_from_data=True)
    kolac.set_categories(popisky)
    kolac.title = "Analýza financí - Koláčový graf"
    ws.add_chart(kolac, "G1")

    # lajny pro datum
    line_ws = wb.create_sheet(title="Line Chart Data")

    # Prepare data for line chart including nested project data
    line_data = [
        (
            item['Název'], 
            item['Název projektu'], 
            item['Datum zahájení'], 
            item['Datum ukončení']
        ) for index, item in dataframe.iterrows() 
        if item['Název projektu'] and item['Datum zahájení'] and item['Datum ukončení']
    ]

    line_df = pd.DataFrame(line_data, columns=['FinanceName', 'ProjectName', 'Startdate', 'Enddate'])
    line_df['Startdate'] = pd.to_datetime(line_df['Startdate'])
    line_df['Enddate'] = pd.to_datetime(line_df['Enddate'])

    for r_idx, row in enumerate(dataframe_to_rows(line_df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            line_ws.cell(row=r_idx, column=c_idx, value=value)

    # Vytvoř linkový graf do excelu
    line_chart = LineChart()
    line_chart.title = "Timeline projektu"
    line_chart.y_axis.title = "Datum"
    line_chart.x_axis.title = "Zacatek/konec"

    for index, row in line_df.iterrows():
        series = Series(values=Reference(line_ws, min_col=3, min_row=index+2, max_col=4, max_row=index+2),
                        title=row['ProjectName'])
        line_chart.series.append(series)

    line_ws.add_chart(line_chart, "E2")

    # Prepare data for Sunburst chart
    sunburst_ws = wb.create_sheet(title="Sunburst Chart Data")

    sunburst_data = [
        (
            item['Typ financí'],
            item['Název projektu'],
            item['Částka']
        ) for index, item in dataframe.iterrows()
    ]

    sunburst_df = pd.DataFrame(sunburst_data, columns=['FinanceType', 'ProjectName', 'Amount'])

    for r_idx, row in enumerate(dataframe_to_rows(sunburst_df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            sunburst_ws.cell(row=r_idx, column=c_idx, value=value)

    # Create Sunburst chart and add it to the new sheet
    sunburst_graf = px.sunburst(sunburst_df, path=['FinanceType', 'ProjectName'], values='Amount', title='Analýza financí - Sunburst graf')
    sunburst_graf.update_traces(textinfo='label+percent entry')
    sunburst_graf.write_image("sunburst_graf.png")
    obrazek = openpyxl.drawing.image.Image("sunburst_graf.png")
    sunburst_ws.add_image(obrazek, 'A1')

    wb.save(excel_soubor)

json_soubor = 'finance_analysis.json'
excel_soubor = 'finance_data.xlsx'

# Načti JSON soubor
data = nacti_json_soubor(json_soubor)

# Převeď JSON data na DataFrame
df = preved_json_na_dataframe(data)

# Ulož DataFrame do Excelu s grafy
uloz_do_excelu(df, excel_soubor)